## Import necessary modules

In [1]:
import os 
import numpy as np
import random

In [2]:
os.chdir('/Users/nicopalm/Documents/Projects/helper')
from plotting.notebook_plotting import *

In [3]:
os.chdir('/Users/nicopalm/Documents/Projects/Multi objective optimization')
os.curdir

'.'

In [4]:
from interfaces.function import Function
from weight_functions.scalar import Scalar
from weight_functions.potence import Potence
from minimizers.slsqp import SLSQP
from minimizers.differential_evolution import DifferentialEvolution

## Define the underlying function

In [23]:
input_dimensions = 8
output_dimensions = 2
starting_point = np.ones(input_dimensions)
bounds = tuple([(0, 1) for _ in range(input_dimensions)])

In [24]:
# Define the function
from pymoo.factory import get_problem
from pymoo.visualization.scatter import Scatter
from pymoo.factory import get_performance_indicator
class ExampleFunction(Function):
    def __call__(self, x):
        self._evaluations.append([
            x,
            get_problem("dtlz2_-1",
                        n_var=input_dimensions,
                        n_obj=output_dimensions).evaluate(x)
        ])
        return get_problem("dtlz2_-1",
                           n_var=input_dimensions,
                           n_obj=output_dimensions).evaluate(x)

In [25]:
# Initialiaze the function
function = ExampleFunction()

## Visualize the function

In [26]:
number_sample_points = 1000
sample_points = [
    np.array([random.uniform(bound[0], bound[1]) for bound in bounds])
    for _ in range(number_sample_points)
]
evaluations_sample_points = [function(point) for point in sample_points]

In [27]:
# Plot function
class ExampleDataFunction(DataFunction):
    def __init__(self, index_fixed):
        self.index_fixed = index_fixed
        self.title = ''

    def __call__(self, index):
        x_dots = [
            point[self.index_fixed] for point in evaluations_sample_points
        ]
        y_dots = [point[index] for point in evaluations_sample_points]
        return [dot_scatter(x_dots=x_dots, y_dots=y_dots)]

In [28]:
interactive_plot(data_function=ExampleDataFunction(index_fixed=0),indices=(0,output_dimensions-1,1),start_index=1)

interactive(children=(IntSlider(value=1, description='index', max=1), Output()), _dom_classes=('widget-interac…

### Visualization of the function/manual tuning of parameter

In [11]:
from ipywidgets import interact

In [12]:
x0 = np.array([0.5,0.5])

y0 = function(x0)

In [13]:
# TODO: add better point found when a dominated points is found 
# index = [index_parameter_1,index_parameter_2,...]
data = [dot_scatter(x_dots=['index 0','index 1'],y_dots=y0),dot_scatter(x_dots=['index 0','index 1'],y_dots=y0)]
fig0 = go.FigureWidget(data=data)
# fixed scaling
fig0.update_yaxes(autorange=False)

#for i in range(input_dimensions):
@interact(index_0=(0, 1, 0.01), index_1=(0, 1, 0.01))
def update(index_0=x0[0], index_1=x0[1]):
    with fig0.batch_update():
        fig0.data[0].y = function(np.array([index_0,index_1]))
    return fig0


interactive(children=(FloatSlider(value=0.5, description='index_0', max=1.0, step=0.01), FloatSlider(value=0.5…

In [14]:
# slider for all design deimensions, 

## Calculate the pareto optima

In [15]:
# Define the weight function
#weight_function = Scalar(theta=np.ones(output_dimensions))
weight_function = Potence(potence=3 * np.ones(output_dimensions))

In [16]:
# Define the minimizer
#minimizer = SLSQP(starting_point=starting_point)
minimizer = DifferentialEvolution(starting_point=starting_point, bounds=bounds)

In [17]:
function._evaluations = [
]  # reset the evaluations (used to approximate the image of the function)
minimizer(lambda x: weight_function(function(x)))

differential_evolution step 1: f(x)= 0.709047
differential_evolution step 2: f(x)= 0.709047
differential_evolution step 3: f(x)= 0.707784
differential_evolution step 4: f(x)= 0.707784
differential_evolution step 5: f(x)= 0.707784
differential_evolution step 6: f(x)= 0.707131
differential_evolution step 7: f(x)= 0.707131
differential_evolution step 8: f(x)= 0.707122
differential_evolution step 9: f(x)= 0.70712
differential_evolution step 10: f(x)= 0.70712
differential_evolution step 11: f(x)= 0.70712
differential_evolution step 12: f(x)= 0.707108
differential_evolution step 13: f(x)= 0.707107
differential_evolution step 14: f(x)= 0.707107
differential_evolution step 15: f(x)= 0.707107


array([0.5000005 , 0.50000033])

## Plot evaluations of sampled points

In [18]:
# Plot function
class ExampleDataFunction(DataFunction):
    def __init__(self, index_fixed):
        self.index_fixed = index_fixed
        self.title = ''

    def __call__(self, index):
        x_dots_eval = [
            point[1][self.index_fixed] for point in function.evaluations
        ]
        y_dots_eval = [point[1][index] for point in function.evaluations]
        return [
            dot_scatter(x_dots=x_dots_eval, y_dots=y_dots_eval),
            dot_scatter(
                x_dots=[function(minimizer.result.x)[self.index_fixed]],
                y_dots=[function(minimizer.result.x)[index]],
                name_dots='Pareto optimum found')
        ]

In [19]:
interactive_plot(data_function=ExampleDataFunction(index_fixed=0),
                 indices=(0, output_dimensions-1, 1),
                 start_index=1)

interactive(children=(IntSlider(value=1, description='index', max=1), Output()), _dom_classes=('widget-interac…

## Look at the results

In [20]:
#function.evaluations

In [21]:
minimizer.result

     fun: 0.7071067811874266
     jac: array([2.60902409e-06, 1.39888100e-06])
 message: 'Optimization terminated successfully.'
    nfev: 492
     nit: 15
 success: True
       x: array([0.5000005 , 0.50000033])

In [22]:
minimizer.number_evaluations_last_call

492